In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import seaborn as sns

2024-01-13 08:19:00.277636: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-13 08:19:00.388486: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-13 08:19:00.390693: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-13 08:19:02.349659: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
df = pd.read_csv('~/MITM-Detection/dataset.csv')

In [3]:

# Display the first few rows of the DataFrame
print(df.head())

# Extract features and labels
X = df.iloc[:, :-1]  # Features (all columns except the last one)
y = df.iloc[:, -1]   # Labels (last column)

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape dữ liệu để phù hợp với đầu vào của Conv1D
X_train = X_train.values.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.values.reshape(X_test.shape[0], X_test.shape[1], 1)

           0  6.666666667  0.1  0.2  0.3
0   8.000000    61.666667   24    0    0
1  18.000000    43.333333   54    0    0
2  23.666667    41.333333   71    0    0
3  37.666667    38.333333   91    0    0
4  52.333333    25.000000   95    0    0


In [4]:
# model = Sequential()
# model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Flatten())
# model.add(Dense(1, activation='sigmoid'))

In [5]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=(X_train.shape[1], 1)))
model.add(Dense(1))
model.compile(loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=2, validation_data=(X_test, y_test))

# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# # Huấn luyện mô hình
# model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

# Dự đoán trên tập kiểm tra
y_pred = model.predict(X_test)
y_pred_binary = (y_pred > 0.5).astype(int)

Epoch 1/100
1179/1179 - 11s - loss: 0.3326 - accuracy: 0.8882 - val_loss: 0.1130 - val_accuracy: 0.9727 - 11s/epoch - 9ms/step
Epoch 2/100
1179/1179 - 5s - loss: 0.1146 - accuracy: 0.9733 - val_loss: 0.1168 - val_accuracy: 0.9742 - 5s/epoch - 4ms/step
Epoch 3/100
1179/1179 - 5s - loss: 0.1239 - accuracy: 0.9746 - val_loss: 0.1853 - val_accuracy: 0.9742 - 5s/epoch - 4ms/step
Epoch 4/100
1179/1179 - 5s - loss: 0.1314 - accuracy: 0.9746 - val_loss: 0.1852 - val_accuracy: 0.9742 - 5s/epoch - 4ms/step
Epoch 5/100
1179/1179 - 5s - loss: 0.1227 - accuracy: 0.9746 - val_loss: 0.1387 - val_accuracy: 0.9742 - 5s/epoch - 4ms/step
Epoch 6/100
1179/1179 - 5s - loss: 0.1347 - accuracy: 0.9746 - val_loss: 0.1281 - val_accuracy: 0.9742 - 5s/epoch - 4ms/step
Epoch 7/100
1179/1179 - 5s - loss: 0.1164 - accuracy: 0.9746 - val_loss: 0.1275 - val_accuracy: 0.9742 - 5s/epoch - 4ms/step
Epoch 8/100
1179/1179 - 5s - loss: 0.1259 - accuracy: 0.9746 - val_loss: 0.1498 - val_accuracy: 0.9742 - 5s/epoch - 4ms/ste

In [6]:
# Tính confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred_binary)
print("Confusion Matrix:")
print(conf_matrix)

Confusion Matrix:
[[6256    0]
 [ 239 2931]]


In [7]:
#Get the evaluation metrics like precision, recall, f1-score
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_binary))


              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6256
           1       1.00      0.92      0.96      3170

    accuracy                           0.97      9426
   macro avg       0.98      0.96      0.97      9426
weighted avg       0.98      0.97      0.97      9426

